In [ ]:
!git clone https://github.com/HoangHao1009/hminiGPT
%cd hminiGPT
!pip install -e .

Cloning into 'hminiGPT'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 121 (delta 28), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (121/121), 38.72 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/hminiGPT
Obtaining file:///content/hminiGPT
  Preparing metadata (setup.py) ... done
  Running setup.py develop for hhhGPT


In [ ]:
pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/packages/vi_core_news_lg-3.6.0/dist/vi_core_news_lg-3.6.0.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vi-core-news-lg: filename=vi_core_news_lg-3.6.0-py3-none-any.whl size=233275663 sha256=a07c519463e40242ecc8c52f6f6d4271347e8a29f07604e06baf4cd95540e371
  Stored in directory: /root/.cache/pip/wheels/bd/c2/22/8dfcbf9006c1be9c5f38dda2e8608eddb2f46c933f174c7581
Successfully built vi-core-news-lg


In [ ]:
pip install pyvi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import spacy
from hGPT.datacreator import DataCreator, Vocabulary, CustomDataset
from hGPT.model import MiniGPT, Trainer, WordVector

In [ ]:
CUDA_LAUNCH_BLOCKING=1
torch.cuda.synchronize()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
#model params
block_size = 128
batch_size = 32
vocab_size = None
n_emb = 512
num_head = 2
n_layer = 2
dropout = 0.3
#trainer params
learning_rate = 0.0001
eval_iters = 100

cuda


In [ ]:
tokenizer = spacy.load('vi_core_news_lg')
train_data = DataCreator(block_size, tokenizer)
val_data = DataCreator(block_size, tokenizer)

#extractPairs Data:
#train_file_path = '/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/10000book_train.txt'
#val_file_path = '/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/10000book_val.txt'
#train_data.extractPairs(train_file_path, 50000)
#val_data.extractPairs(val_file_path, 5000)
#train_data.csvwrite('/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/trainGPTdata.csv', ',')
#val_data.csvwrite('/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/valGPTdata.csv', ',')

#If you've extracted:
train_data.csvread('/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/10000book/trainGPTdata.csv', ',')
val_data.csvread('/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/10000book/valGPTdata.csv', ',')

Reading CSV: 250000rows [02:42, 1534.48rows/s]
Reading CSV: 50000rows [00:30, 1631.05rows/s]


In [ ]:
voc = Vocabulary()
train_dataset = CustomDataset(train_data.pairs, voc, device, min_count = 5, type = 'train')
val_dataset = CustomDataset(val_data.pairs, voc, device, type = 'val')

Keep 60644 in total 165469 = 36.649765%


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size, shuffle = True)

In [ ]:
model = MiniGPT(voc.num_words, n_emb, block_size, num_head, n_layer, dropout, device)
m = model.to(device)

In [ ]:
model.cuda

<bound method Module.cuda of MiniGPT(
  (token_embedding): Embedding(60646, 512)
  (position_embedding): Embedding(128, 512)
  (trans_blocks): Sequential(
    (0): TransformerBlock(
      (self_attention): MultiHeadAttention(
        (heads): ModuleList(
          (0-1): 2 x HeadAttention(
            (key): Linear(in_features=512, out_features=256, bias=False)
            (query): Linear(in_features=512, out_features=256, bias=False)
            (value): Linear(in_features=512, out_features=256, bias=False)
            (dropout): Dropout(p=0.3, inplace=False)
          )
        )
        (proj): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
      )
      (feedforward): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): ReLU()
        (2): Linear(in_features=2048, out_features=512, bias=True)
        (3): Dropout(p=0.3, inplace=False)
      )
      (ln1): LayerNorm((512,), eps=1e-05, eleme

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [ ]:
trainer = Trainer(train_dataloader, val_dataloader, model, optimizer, eval_iters)
trainer.start_training(1000)

Step 0: Train loss: 11.106707572937012/ Val loss: 11.106467247009277
Step 50: Train loss: 7.73062801361084/ Val loss: 7.720537185668945
Step 100: Train loss: 7.1295166015625/ Val loss: 7.110549449920654
Step 150: Train loss: 7.048479080200195/ Val loss: 7.0374603271484375
Step 200: Train loss: 6.965098857879639/ Val loss: 6.954097270965576
Step 250: Train loss: 6.877810001373291/ Val loss: 6.876185417175293
Step 300: Train loss: 6.804450035095215/ Val loss: 6.8037848472595215
Step 350: Train loss: 6.7003278732299805/ Val loss: 6.718346118927002
Step 400: Train loss: 6.641290187835693/ Val loss: 6.643800258636475
Step 450: Train loss: 6.564385414123535/ Val loss: 6.553297519683838
Step 500: Train loss: 6.525985240936279/ Val loss: 6.504528045654297
Step 550: Train loss: 6.4490275382995605/ Val loss: 6.453876972198486
Step 600: Train loss: 6.4017558097839355/ Val loss: 6.395174503326416
Step 650: Train loss: 6.376269340515137/ Val loss: 6.328586578369141
Step 700: Train loss: 6.315573692

In [ ]:
trainer.start_training(1000)

Step 0: Train loss: 6.131241321563721/ Val loss: 6.134433746337891
Step 50: Train loss: 6.117010593414307/ Val loss: 6.0885443687438965
Step 100: Train loss: 6.083968639373779/ Val loss: 6.066431999206543
Step 150: Train loss: 6.0530619621276855/ Val loss: 6.053677558898926
Step 200: Train loss: 6.025522232055664/ Val loss: 6.035211086273193
Step 250: Train loss: 6.01740837097168/ Val loss: 6.008098602294922
Step 300: Train loss: 6.009273529052734/ Val loss: 6.016934871673584
Step 350: Train loss: 5.994997024536133/ Val loss: 5.976059436798096
Step 400: Train loss: 5.975733757019043/ Val loss: 5.976858615875244
Step 450: Train loss: 5.951131820678711/ Val loss: 5.9807868003845215
Step 500: Train loss: 5.933539867401123/ Val loss: 5.942166805267334
Step 550: Train loss: 5.914873123168945/ Val loss: 5.928574085235596
Step 600: Train loss: 5.906093597412109/ Val loss: 5.931536674499512
Step 650: Train loss: 5.884488582611084/ Val loss: 5.90078067779541
Step 700: Train loss: 5.885556697845

In [ ]:
trainer.start_training(2000)

Step 0: Train loss: 5.813185214996338/ Val loss: 5.821667671203613
Step 50: Train loss: 5.780452251434326/ Val loss: 5.822023868560791
Step 100: Train loss: 5.784909725189209/ Val loss: 5.823739528656006
Step 150: Train loss: 5.767423629760742/ Val loss: 5.803229808807373
Step 200: Train loss: 5.767155647277832/ Val loss: 5.795984268188477
Step 250: Train loss: 5.73458194732666/ Val loss: 5.7717814445495605
Step 300: Train loss: 5.7463459968566895/ Val loss: 5.770205020904541
Step 350: Train loss: 5.732873916625977/ Val loss: 5.754355430603027
Step 400: Train loss: 5.7281174659729/ Val loss: 5.742471694946289
Step 450: Train loss: 5.7178826332092285/ Val loss: 5.741108417510986
Step 500: Train loss: 5.683190822601318/ Val loss: 5.743659019470215
Step 550: Train loss: 5.687924861907959/ Val loss: 5.739001274108887
Step 600: Train loss: 5.665472030639648/ Val loss: 5.72268533706665
Step 650: Train loss: 5.6624674797058105/ Val loss: 5.71302604675293
Step 700: Train loss: 5.65720081329345

In [ ]:
trainer.start_training(1000)

Step 0: Train loss: 5.478590488433838/ Val loss: 5.568547248840332
Step 50: Train loss: 5.468776226043701/ Val loss: 5.559711933135986
Step 100: Train loss: 5.45071268081665/ Val loss: 5.548589706420898
Step 150: Train loss: 5.455460071563721/ Val loss: 5.544882774353027
Step 200: Train loss: 5.46367073059082/ Val loss: 5.526522159576416
Step 250: Train loss: 5.445815563201904/ Val loss: 5.52263069152832
Step 300: Train loss: 5.460038661956787/ Val loss: 5.526165962219238
Step 350: Train loss: 5.448351860046387/ Val loss: 5.510110378265381
Step 400: Train loss: 5.449069976806641/ Val loss: 5.524076461791992
Step 450: Train loss: 5.427732467651367/ Val loss: 5.4994120597839355
Step 500: Train loss: 5.43520975112915/ Val loss: 5.518751621246338
Step 550: Train loss: 5.439608097076416/ Val loss: 5.512999057769775
Step 600: Train loss: 5.435047626495361/ Val loss: 5.509336948394775
Step 650: Train loss: 5.434916973114014/ Val loss: 5.507773399353027
Step 700: Train loss: 5.41361141204834/ 

In [ ]:
trainer.save_model(
    voc,
    '/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/10000book/saveGPTmodel5000.pt'
)

In [ ]:
trainer.start_training(5000)
trainer.save_model(
    voc,
    '/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/10000book/saveGPTmodel10000.pt'
)
print('Saving succesfully')

Step 0: Train loss: 5.382081508636475/ Val loss: 5.483151912689209
Step 50: Train loss: 5.389092445373535/ Val loss: 5.476998329162598
Step 100: Train loss: 5.375117301940918/ Val loss: 5.470396041870117
Step 150: Train loss: 5.382370471954346/ Val loss: 5.476761341094971
Step 200: Train loss: 5.378304481506348/ Val loss: 5.4546356201171875
Step 250: Train loss: 5.357542514801025/ Val loss: 5.460648059844971
Step 300: Train loss: 5.360203742980957/ Val loss: 5.45670223236084
Step 350: Train loss: 5.3553466796875/ Val loss: 5.462347507476807
Step 400: Train loss: 5.360458850860596/ Val loss: 5.459204196929932
Step 450: Train loss: 5.35096549987793/ Val loss: 5.454953670501709
Step 500: Train loss: 5.343104362487793/ Val loss: 5.445811748504639
Step 550: Train loss: 5.354085922241211/ Val loss: 5.443505764007568
Step 600: Train loss: 5.334701061248779/ Val loss: 5.4496870040893555
Step 650: Train loss: 5.331259727478027/ Val loss: 5.447030067443848
Step 700: Train loss: 5.342055797576904